In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

Dataset can be found here: http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

If you have any questions about the implementation or extending this project and capturing your own data please reach me at matt@tractionhealth.com

In [ ]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    
    
    return np.transpose(np.array(X_signals), (1, 2, 0))

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    # Substract 1 to each output class for friendly 0-based indexing 
    return y_ 

In [ ]:
train_features_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
test_features_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

train_features = load_X(train_features_signals_paths)
test_features = load_X(test_features_signals_paths)

In [ ]:
y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

train_labels = load_y(y_train_path)
test_labels = load_y(y_test_path)

In [ ]:
train_features.shape, test_features.shape, train_labels.shape, test_labels.shape

((7352, 128, 9), (2947, 128, 9), (7352, 1), (2947, 1))

In [ ]:
# Quickly one-hot encoding the labels
df_train_labels = pd.DataFrame(train_labels, columns=["class"])
df_train_labels = pd.get_dummies(df_train_labels['class'])
train_labels = df_train_labels.to_numpy()

df_test_labels = pd.DataFrame(test_labels, columns=['class'])
df_test_labels = pd.get_dummies(df_test_labels['class'])
test_labels = df_test_labels.to_numpy()

In [ ]:
test_labels

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [ ]:
 ds_train = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
 ds_test = tf.data.Dataset.from_tensor_slices((test_features, test_labels))

In [ ]:
ds_train_batch = ds_train.batch(64)
ds_test_batch = ds_test.batch(64)

In [ ]:
model_2 = keras.Sequential([
                            keras.layers.LSTM(6, input_shape=(128, 9)),
                            #keras.layers.Dropout(0.2)
                            #keras.layers.LSTM(128)
                            keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model_2.compile(loss=keras.losses.CategoricalCrossentropy(),
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])

In [ ]:
model_2_hist = model_2.fit(ds_train_batch, batch_size=64, epochs=25)

Epoch 1/25
115/115 [==============================] - 7s 40ms/step - loss: 1.7213 - accuracy: 0.2897
Epoch 2/25
115/115 [==============================] - 5s 40ms/step - loss: 1.6020 - accuracy: 0.3573
Epoch 3/25
115/115 [==============================] - 5s 40ms/step - loss: 1.4561 - accuracy: 0.4249
Epoch 4/25
115/115 [==============================] - 5s 40ms/step - loss: 1.2985 - accuracy: 0.5445
Epoch 5/25
115/115 [==============================] - 5s 40ms/step - loss: 1.1876 - accuracy: 0.5817
Epoch 6/25
115/115 [==============================] - 5s 41ms/step - loss: 1.1043 - accuracy: 0.6153
Epoch 7/25
115/115 [==============================] - 5s 40ms/step - loss: 1.0257 - accuracy: 0.6436
Epoch 8/25
115/115 [==============================] - 5s 40ms/step - loss: 1.0331 - accuracy: 0.6217
Epoch 9/25
115/115 [==============================] - 5s 40ms/step - loss: 0.9594 - accuracy: 0.6578
Epoch 10/25
115/115 [==============================] - 5s 40ms/step - loss: 0.9542 - accura

In [ ]:
model_3 = keras.Sequential([
                            keras.layers.LSTM(6, input_shape=(128, 9), return_sequences=True),
                            keras.layers.Dropout(0.2),
                            keras.layers.LSTM(6),
                            keras.layers.Dense(6, activation='softmax')
])

model_3.compile(loss=keras.losses.CategoricalCrossentropy(),
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])

model_3_hist = model_3.fit(ds_train_batch, batch_size=64, epochs=25)

Epoch 1/25
115/115 [==============================] - 13s 82ms/step - loss: 1.7303 - accuracy: 0.3649
Epoch 2/25
115/115 [==============================] - 10s 83ms/step - loss: 1.4515 - accuracy: 0.4795
Epoch 3/25
115/115 [==============================] - 9s 81ms/step - loss: 1.3014 - accuracy: 0.5248
Epoch 4/25
115/115 [==============================] - 9s 82ms/step - loss: 1.2233 - accuracy: 0.5355
Epoch 5/25
115/115 [==============================] - 9s 81ms/step - loss: 1.1677 - accuracy: 0.5502
Epoch 6/25
115/115 [==============================] - 9s 81ms/step - loss: 1.0981 - accuracy: 0.5896
Epoch 7/25
115/115 [==============================] - 9s 82ms/step - loss: 1.1079 - accuracy: 0.5362
Epoch 8/25
115/115 [==============================] - 9s 82ms/step - loss: 1.0413 - accuracy: 0.5661
Epoch 9/25
115/115 [==============================] - 9s 81ms/step - loss: 1.1454 - accuracy: 0.4909
Epoch 10/25
115/115 [==============================] - 9s 81ms/step - loss: 1.0023 - accu

In [ ]:
model_4 = keras.Sequential([
                            keras.layers.LSTM(6, input_shape=(128, 9), return_sequences=True),
                            keras.layers.Dropout(0.2),
                            keras.layers.LSTM(6, return_sequences=True),
                            keras.layers.Dropout(0.2),
                            keras.layers.LSTM(6),
                            keras.layers.Dense(6, activation='softmax')
])

model_4.compile(loss=keras.losses.CategoricalCrossentropy(),
             optimizer=keras.optimizers.Adam(learning_rate=0.002),
             metrics=['accuracy'])

model_4_hist = model_4.fit(ds_train_batch, batch_size=64, epochs=25)

Epoch 1/25
115/115 [==============================] - 20s 133ms/step - loss: 1.5834 - accuracy: 0.3542
Epoch 2/25
115/115 [==============================] - 15s 133ms/step - loss: 1.2982 - accuracy: 0.4514
Epoch 3/25
115/115 [==============================] - 15s 133ms/step - loss: 1.2494 - accuracy: 0.4709
Epoch 4/25
115/115 [==============================] - 15s 134ms/step - loss: 1.2760 - accuracy: 0.4607
Epoch 5/25
115/115 [==============================] - 15s 132ms/step - loss: 1.1491 - accuracy: 0.5657
Epoch 6/25
115/115 [==============================] - 15s 133ms/step - loss: 1.0350 - accuracy: 0.5344
Epoch 7/25
115/115 [==============================] - 15s 133ms/step - loss: 0.9304 - accuracy: 0.5097
Epoch 8/25
115/115 [==============================] - 15s 133ms/step - loss: 0.8586 - accuracy: 0.5394
Epoch 9/25
115/115 [==============================] - 15s 134ms/step - loss: 0.8494 - accuracy: 0.5566
Epoch 10/25
115/115 [==============================] - 15s 134ms/step - l